# Example Scenario 1: Generating embeddings for ConceptNet nodes

*Alice wishes to import the English subset of ConceptNet in KGTK format. Then, she would extract a subset of ConceptNet where two concepts are connected with a precise semantic relation, like `Causes` or `UsedFor` (as opposed to weaker relations like `/r/RelatedTo`). Text embeddings would be computed for all nodes in this subset, and saved in a file called `emb.txt`.*

## Preparation

To run this notebook, Alice would need the ConceptNet graph file. We will work with the latest ConceptNet, v5.7.0. Presumably, this file is not present on Alice's laptop, so we need to download and unpack it first (note: mac users might need to install `wget` first: `brew install wget`):

In [ ]:
%%bash
wget https://s3.amazonaws.com/conceptnet/downloads/2019/edges/conceptnet-assertions-5.7.0.csv.gz

In [ ]:
%%bash
gunzip conceptnet-assertions-5.7.0.csv.gz

## Implementation in KGTK

We will select the relevant edges from ConceptNet and sort them (note that we extract three more relations which will be used to extract labels, descriptions, and inheritance by our embedding generator below).

Then we compute text embeddings. For demonstration purposes, we will compute embeddings based on the first 30k edges.

In [1]:
%%bash
kgtk import_conceptnet --english_only conceptnet-assertions-5.7.0.csv / \
            filter -p " ; /r/Causes,/r/UsedFor,/r/Synonym,/r/DefinedAs,/r/IsA ; " / sort -c 1,2,3 \
            | head -30000 |
            kgtk text_embedding --debug --embedding-projector-metadata-path none \
                    --embedding-projector-metadata-path none \
                    --label-properties "/r/Synonym" \
                    --isa-properties "/r/IsA" \
                    --description-properties "/r/DefinedAs" \
                    --property-value "/r/Causes" "/r/UsedFor" \
                    --has-properties "" \
                    -f kgtk_format \
                    --output-format kgtk_format \
                    --use-cache \
                    --model bert-large-nli-cls-token \
                    > emb.txt                 

100%|██████████| 19570/19570 [37:35<00:00,  8.68it/s] 


Let's inspect the result, by printing the first embedding:

In [2]:
!head -2 emb.txt

node	property	value
/c/en/astragalus_glycyphyllos	text_embedding	-0.18993971,-0.18467772,0.97278947,-1.3457584,0.522082,-0.08496471,-0.36786398,0.61815584,0.4998474,0.39584792,0.45506585,-0.32172498,0.39399725,0.21073538,-0.5026435,0.2623347,-0.69918454,-0.346811,-1.4539826,1.152729,1.3615526,0.18331644,-1.4889344,0.9622106,-0.36267373,0.031175017,-0.81846493,0.2910592,0.9767509,0.16649616,-0.5486471,-1.0536056,0.14126939,-0.686479,-0.4030986,-0.07771198,-0.48865426,0.38048646,-0.04203824,-0.47893983,0.072292015,-0.37736234,0.39158025,-0.10181721,0.1688336,-0.4667816,-0.10422518,-0.004381031,0.16471443,0.41520813,-0.1718134,-0.3819918,1.5958356,0.33272198,-0.23313947,-0.5774998,1.2301497,-0.13966326,-0.49593198,1.1411607,0.2562833,0.42478192,-0.21777846,-1.1025244,0.27039,-0.31927243,0.25055987,-0.02119275,0.6250978,-0.23635834,-0.608614,0.074760474,-0.17777503,0.06811429,-0.8170296,0.77169275,-0.33680287,-0.4864985,0.479845,1.0216551,0.3938682,0.010071404,0.45509982,-0.5486847,-0.8565